# 1. Info

Notebook with all the code needed to solve the homework for the week number 8 of the machine learning zoomcamp.

## Instal the required libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import Sequential

## Geting the data

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. For this, we will use the "Bee or Wasp?" dataset that was obtained from Kaggle and slightly rebuilt.

You can download the dataset for this homework from here:

In [2]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip

## Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps.

The dataset contains separate folders for training and test sets.

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be (150, 150, 3)
* Next, create a convolutional layer (Conv2D):
    * Use 32 filters
    * Kernel size should be (3, 3) (that's the size of the filter)
    * Use 'relu' as activation
* Reduce the size of the feature map with max pooling (MaxPooling2D)
    * Set the pooling size to (2, 2)
* Turn the multi-dimensional result into vectors using a Flatten layer
* Next, add a Dense layer with 64 neurons and 'relu' activation
* Finally, create the Dense layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:

* SGD(lr=0.002, momentum=0.8)

For clarification about kernel size and max pooling, check Office Hours.

In [3]:
model = Sequential([    
    keras.Input(shape=(150, 150, 3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"), 
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=64, activation="relu"),
    keras.layers.Dense(units=1, activation="sigmoid")
])

optimizer = keras.optimizers.SGD(
    learning_rate=0.002, 
    momentum=0.8
)

model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Question 1

Since we have a binary classification problem, what is the best loss function for us?

* mean squared error
* binary crossentropy
* categorical crossentropy
* cosine similarity


Answer = binary crossentropy

# Question 2

What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.

* 1
* 65
* 896
* 11214912

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

Answer = 896

# Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
    ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
* Use batch_size=20
* Use shuffle=True for both training and test sets.

For training use .fit() with the following params:

```python
    model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator
    )
```

In [5]:
batch_size = 20

train_gen = ImageDataGenerator(rescale=1./255)
val_gen   = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    "./data/train/", 
    target_size=(150, 150), 
    batch_size=batch_size,
    shuffle=True,
    class_mode="binary"
)

val_ds = val_gen.flow_from_directory(
    "./data/test/", 
    target_size=(150, 150), 
    batch_size=batch_size,
    shuffle=True,
    class_mode="binary"
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
184/184 [==============================] - 11s 61ms/step - loss: 0.6775 - accuracy: 0.5611 - val_loss: 0.6352 - val_accuracy: 0.6155
Epoch 2/10
184/184 [==============================] - 10s 55ms/step - loss: 0.6262 - accuracy: 0.6435 - val_loss: 0.6001 - val_accuracy: 0.6296
Epoch 3/10
184/184 [==============================] - 11s 58ms/step - loss: 0.5721 - accuracy: 0.7215 - val_loss: 0.5575 - val_accuracy: 0.7004
Epoch 4/10
184/184 [==============================] - 11s 57ms/step - loss: 0.5347 - accuracy: 0.7449 - val_loss: 0.5474 - val_accuracy: 0.7200
Epoch 5/10
184/184 [==============================] - 11s 57ms/step - loss: 0.5115 - accuracy: 0.7533 - val_loss: 0.5677 - val_accuracy: 0.6950
Epoch 6/10
184/184 [==============================] - 11s 60ms/step - loss: 0.4928 - accuracy: 0.7735 - val_loss: 0.5327 - val_accuracy: 0.7505
Epoch 7/10
184/184 [==============================] - 11s 58ms/step - loss: 0.4657 - accuracy: 0.7955 - val_loss: 0.5157 - val_accuracy:

# Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80

In [7]:
hist = history.history

In [8]:
np.median(hist['accuracy'])

0.7633940875530243

Answer: 0.763

# Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131

In [9]:
np.std(hist["loss"])

0.09234823155019067

Answer: 0.092

# Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

* rotation_range=50,
* width_shift_range=0.1,
* height_shift_range=0.1,
* zoom_range=0.1,
* horizontal_flip=True,
* fill_mode='nearest'

In [10]:
batch_size = 20

train_gen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    "./data/train/", 
    target_size=(150, 150), 
    batch_size=batch_size,
    shuffle=True,
    class_mode="binary"
)

val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    "./data/test/", 
    target_size=(150, 150), 
    batch_size=batch_size,
    shuffle=True,
    class_mode="binary"
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


# Question 5

Let's train our model for 10 more epochs using the same code as previously.

Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

In [11]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
184/184 [==============================] - 14s 76ms/step - loss: 0.4960 - accuracy: 0.7683 - val_loss: 0.4958 - val_accuracy: 0.7691
Epoch 2/10
184/184 [==============================] - 14s 74ms/step - loss: 0.4874 - accuracy: 0.7710 - val_loss: 0.4776 - val_accuracy: 0.7614
Epoch 3/10
184/184 [==============================] - 15s 79ms/step - loss: 0.4844 - accuracy: 0.7822 - val_loss: 0.5204 - val_accuracy: 0.7549
Epoch 4/10
184/184 [==============================] - 15s 82ms/step - loss: 0.4794 - accuracy: 0.7811 - val_loss: 0.4860 - val_accuracy: 0.7723
Epoch 5/10
184/184 [==============================] - 17s 90ms/step - loss: 0.4667 - accuracy: 0.7846 - val_loss: 0.5635 - val_accuracy: 0.7440
Epoch 6/10
184/184 [==============================] - 18s 97ms/step - loss: 0.4698 - accuracy: 0.7868 - val_loss: 0.4704 - val_accuracy: 0.7963
Epoch 7/10
184/184 [==============================] - 17s 91ms/step - loss: 0.4690 - accuracy: 0.7914 - val_loss: 0.4946 - val_accuracy:

In [12]:
np.mean(hist["val_loss"])

0.5548043847084045

Answer: 0.55

# Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98

In [13]:
np.mean(hist["val_accuracy"][5:])

0.7516339898109436

Answer: 0.75